In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 16.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612229 sha256=9dd1d612e38064ed39a5ac567a232ad051c9ae84db364d3c20c07644632f7a01
  Stored in directory: /root/.cache/pip/wheels/19/b0/c8/6cb894117070e130fc44352c2a13f15b6c27e440d04a84fb48
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("Boston Housing Linear Regression example")\
    .getOrCreate()

spark

In [4]:
# 데이터 다운로드
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv

--2023-06-15 07:37:55--  https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 3.5.81.18, 3.5.76.103, 3.5.80.204, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|3.5.81.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/csv]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K  --.-KB/s    in 0.01s   

2023-06-15 07:37:56 (2.72 MB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [5]:
data = spark.read.csv('./boston_housing.csv', header=True, inferSchema=True)

data.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [8]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|              crim|                zn|             indus|              chas|                nox|                rm|               age|              dis|              rad|               tax|           ptratio|                 b|             lstat|              medv|
+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|               506|               506|               506|               506|                506|               506|               506|              506|              

In [6]:
data.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

In [7]:
from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data_2 = assembler.transform(data)

data_2.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

In [9]:
# 훈련셋 테스트셋 나누기
train, test = data_2.randomSplit([0.7, 0.3])

In [10]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol="features", labelCol="medv")
model = algo.fit(train)

In [11]:
# 성능 측정
evaluation_summary = model.evaluate(test)

evaluation_summary.meanAbsoluteError

3.282709139297446

In [12]:
evaluation_summary.rootMeanSquaredError

4.5731559339665875

In [13]:
evaluation_summary.r2

0.7735898173736925

In [14]:
# 예측값 확인
prediction = model.transform(test)

prediction.select(prediction.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|32.7|[0.01301,35.0,1.5...| 30.09605529481854|
|35.4|[0.01311,90.0,1.2...|30.872904664663942|
|24.5|[0.01501,80.0,2.0...| 27.41530369751836|
|44.0|[0.01538,90.0,3.7...| 36.67185442935007|
|33.0|[0.01951,17.5,1.3...| 23.37537516870863|
|20.1|[0.01965,80.0,1.7...|19.684576253495724|
|50.0|[0.02009,95.0,2.6...|42.123205786453255|
|42.3|[0.02177,82.5,2.0...| 36.59030577656705|
|16.5|[0.02498,0.0,1.89...| 22.30959438190237|
|18.5|[0.03041,0.0,5.19...|19.405967062508438|
|19.4|[0.03466,35.0,6.0...|  23.4198540670511|
|27.9|[0.03615,80.0,4.9...|31.186373314284015|
|24.8|[0.03659,25.0,4.8...|25.856957525616643|
|35.4|[0.03705,20.0,3.3...| 34.04535615483184|
|28.0|[0.04113,25.0,4.8...|28.207642676797047|
|24.8|[0.04297,52.5,5.3...|26.652953244483736|
|23.9|[0.04462,25.0,4.8...|26.366681365238236|
|20.6|[0.04527,0.0,11.9...|22.130821083874604|
|19.8|[0.0454

In [15]:
model.save('boston_housing_model')

In [17]:
!ls boston_housing_model

data  metadata


In [18]:
from pyspark.ml.regression import LinearRegressionModel

loaded_model = LinearRegressionModel.load('./boston_housing_model')
prediction2 = loaded_model.transform(test)

In [19]:
prediction2.select(prediction2.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|32.7|[0.01301,35.0,1.5...| 30.09605529481854|
|35.4|[0.01311,90.0,1.2...|30.872904664663942|
|24.5|[0.01501,80.0,2.0...| 27.41530369751836|
|44.0|[0.01538,90.0,3.7...| 36.67185442935007|
|33.0|[0.01951,17.5,1.3...| 23.37537516870863|
|20.1|[0.01965,80.0,1.7...|19.684576253495724|
|50.0|[0.02009,95.0,2.6...|42.123205786453255|
|42.3|[0.02177,82.5,2.0...| 36.59030577656705|
|16.5|[0.02498,0.0,1.89...| 22.30959438190237|
|18.5|[0.03041,0.0,5.19...|19.405967062508438|
|19.4|[0.03466,35.0,6.0...|  23.4198540670511|
|27.9|[0.03615,80.0,4.9...|31.186373314284015|
|24.8|[0.03659,25.0,4.8...|25.856957525616643|
|35.4|[0.03705,20.0,3.3...| 34.04535615483184|
|28.0|[0.04113,25.0,4.8...|28.207642676797047|
|24.8|[0.04297,52.5,5.3...|26.652953244483736|
|23.9|[0.04462,25.0,4.8...|26.366681365238236|
|20.6|[0.04527,0.0,11.9...|22.130821083874604|
|19.8|[0.0454